### Bibliotecas Utilizadas

In [41]:
import sys
import os

# Adiciona a pasta pai ao sys.path
sys.path.append(os.path.abspath(".."))
# Garante que a própria pasta atual esteja no caminho
sys.path.append(os.getcwd())

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from pyautocad import Autocad, APoint
from comtypes import npsupport
npsupport.enable()
from main import *
from materiais import *
from v_p_cantoneira_flex import *
from funcoes_figuras_autocad import *


### Dimensionamento da ligação

#### Tipos de Aço escolhidos para o perfil

In [42]:
aço_escolhido = ASTM_A572

aço_escolhido_cantoneira = ASTM_A36

#### Propriedades geométricas do Perfil

In [43]:
perfil_escolhido = W_300x79_0
perfil_escolhido.inercias()
perfil_escolhido.material(aço_escolhido)

#### Propriedades  dos Parafusos

In [44]:
parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =2)
diametros = diametros_A325
#Escolha da quantidade de parafusos
n_p_min=1
n_p_max = np.floor((perfil_escolhido.h_w-2*30)/67.5)  # 6.5 é a média do espaçamento entre os parafusos
N_parafusos = 3 

#### Esforços solicitantes e solução

In [45]:
V = 186 #kN

T = 73 #kN


S = curva_interacao2(T,V,aço_escolhido_cantoneira,perfil_escolhido,parafuso,diametros,N_parafusos,gamma)

In [46]:
c = S[0]
ver_parafuso = c[-1]
ver_chapa = c[-2]
cantoneira = c[-3]
parafuso_escolhido = c[-4]

In [47]:
parafuso_escolhido.diametro_mm

12.7

### Criação do desenho

In [48]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Desenho4.dwg


In [58]:
limpar_desenho(acad)

Desenho completamente limpo!


#### Gerar coordenadas dos hexágonos

In [59]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

#### Desenhar a cantoneira

In [60]:
objetos_s_cantoneira = desenhar_s_cantoneira(acad, cantoneira, ver_chapa)

#### Desenhar os parafusos do plano XZ

In [61]:
objetos_p1_cantoneira = []   
 # === Parafusos e hexágonos ===
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]
    z_centro = ver_parafuso.iat[i, 3]

    # Face do hexágono em X
    obj1 = acad.model.AddCircle(APoint(x_centro, z_centro, -y_centro), parafuso.diametro_mm / 2)
    obj1.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
    objetos_p1_cantoneira.append(obj1)

    # Face traseira em X
    obj2 = acad.model.AddCircle(APoint(x_centro, z_centro, 0), parafuso.diametro_mm / 2)
    obj2.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
    objetos_p1_cantoneira.append(obj2)

    # Hexágono desenhado com linhas
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, z_centro, y_centro)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(hexagono_transladado[j][0], hexagono_transladado[j][1], -cantoneira.t_mm)
        p2 = APoint(hexagono_transladado[j + 1][0], hexagono_transladado[j + 1][1], -cantoneira.t_mm)

        linha = acad.model.AddLine(p1, p2)
        linha.Rotate3D(APoint(0, 0, 0), APoint(1, 0, 0), math.radians(90))
        objetos_p1_cantoneira.append(linha)

#### Desenhar os parafusos do plano XY

In [62]:
objetos_p2_cantoneira = []
# === Parafusos e hexágonos ===
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 2]
    y_centro = ver_parafuso.iat[i, 1]   #Muda a tabela considerando agora os parafusos do outro plano
    z_centro = ver_parafuso.iat[i, 3]

    # Face do hexágono em X
    obj1 = acad.model.AddCircle(APoint(z_centro, y_centro, -x_centro), parafuso.diametro_mm / 2)
    obj1.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
    objetos_p2_cantoneira.append(obj1)

    # Face traseira em X
    obj2 = acad.model.AddCircle(APoint(z_centro, y_centro, 0), parafuso.diametro_mm / 2)
    obj2.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
    objetos_p2_cantoneira.append(obj2)

    # Hexágono desenhado com linhas
    hexagono_transladado = transladar_pontos(pontos_hexagono, z_centro, y_centro, -y_centro)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(hexagono_transladado[j][0], hexagono_transladado[j][1], -cantoneira.t_mm)
        p2 = APoint(hexagono_transladado[j + 1][0], hexagono_transladado[j + 1][1], -cantoneira.t_mm)

        linha = acad.model.AddLine(p1, p2)
        linha.Rotate3D(APoint(0, 0, 0), APoint(0, 1, 0), math.radians(-90))
        objetos_p2_cantoneira.append(linha)

#### Desenhar seção das cantoneiras

In [63]:
# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 10, perfil_escolhido.t_w/2, (perfil_escolhido.h-cantoneira.comprimento)/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

# Aplica a translação a todos os objetos na lista
for obj in objetos_s_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))
for obj in objetos_p2_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))
for obj in objetos_p1_cantoneira:
    obj.Move(APoint(0,0,0),vetor) 
    obj.Mirror(APoint(1, 0, 0), APoint(0, 0, 0))

#### Desenhar seção do perfil

In [64]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_escolhido, posicao_x=-perfil_escolhido.b_f/2, posicao_y=-perfil_escolhido.h/2, altura_z=0)

# Rotacionar apenas a seção do perfil:
for obj in objetos_secao_perfil:
    obj.Rotate3D(APoint(0, 0, 0), APoint(1,0, 0), math.radians(90))
    obj.Rotate3D(APoint(0, 0, 0), APoint(0,0, 1), math.radians(90))

# Vetor de translação (exemplo: mover 100 mm no eixo X)
dx, dy, dz = 0,0,perfil_escolhido.h/2  # ajuste aqui conforme necessário

# Aponta o vetor de deslocamento
vetor = APoint(dx, dy, dz)

for obj in objetos_secao_perfil:
    obj.Move(APoint(0,0,0),vetor)